Questo notebook utilizza un modello CUT pre-addestrato per trasformare le immagini del dominio **Reale** nello stile del dominio **Sintetico**.

**Prerequisiti:**
1. Avere il Dataset delle immagini caricato (es. `EGO-CH-OBJ-SEG`).
2. Avere il Dataset dei pesi caricato (il file `.pth` ottenuto dal training).

In [ ]:
import os
import shutil

%cd /kaggle/working
if os.path.exists("benchmarking-generative-models-for-domain-adaptation"):
    shutil.rmtree("benchmarking-generative-models-for-domain-adaptation")


print("Clono la repository...")
!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git
%cd benchmarking-generative-models-for-domain-adaptation


print("Installo librerie...")
!pip install -q dominate visdom

In [ ]:
# Patch per il bug noto di argparse in cut_model.py
target_file = "./models/cut/models/cut_model.py"

if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    
    new_content = content.replace("choices='(CUT, cut, FastCUT, fastcut)'", "choices=['CUT', 'cut', 'FastCUT', 'fastcut']")
    
    with open(target_file, "w") as f:
        f.write(new_content)
    print("Patch applicata correttamente.")
else:
    print("Errore: File cut_model.py non trovato.")

In [ ]:
import os
import shutil

# --- 1. CONFIGURAZIONE INPUT (IMMAGINI REALI) ---
# Percorso delle immagini da trasformare (Input)
KAGGLE_REAL_TEST = "/kaggle/input/ego-ch-obj-seg/EGO-CH/real/train/frames"

# Destinazione: testA è la cartella che CUT guarda per l'inferenza
DEST_DIR = "./models/cut/datasets/ego_adaptation"

if os.path.exists(DEST_DIR): shutil.rmtree(DEST_DIR)
os.makedirs(os.path.join(DEST_DIR, "testA"), exist_ok=True)
os.makedirs(os.path.join(DEST_DIR, "testB"), exist_ok=True) # Serve vuota per compatibilità

print("Collego le immagini di input...")
!ln -s "{KAGGLE_REAL_TEST}"/* "{DEST_DIR}/testA/"


# --- 2. CONFIGURAZIONE PESI (CHECKPOINTS) ---
# MODIFICA QUESTO PERCORSO con il nome del tuo dataset dei pesi su Kaggle
# Esempio: "/kaggle/input/ego-cut-weights-epoch40/ego_cut_kaggle"
KAGGLE_WEIGHTS = "/kaggle/input/ego-cut-weights/ego_cut_kaggle" 

CHECKPOINT_DIR = "./models/cut/checkpoints/ego_cut_kaggle"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"Copio i pesi da {KAGGLE_WEIGHTS}...")
!cp -r "{KAGGLE_WEIGHTS}"/* "{CHECKPOINT_DIR}/"

print("Setup completato!")

In [ ]:
%cd models/cut

print("Avvio Generazione (Domain Adaptation)...")
print("Questa operazione potrebbe richiedere alcuni minuti a seconda del numero di immagini.")

!python test.py \
  --dataroot ./datasets/ego_adaptation \
  --name ego_cut_kaggle \
  --CUT_mode CUT \
  --phase test \
  --epoch 40 \
  --num_test 0 \
  --no_html

print("Generazione completata!")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random
import glob

# Percorsi risultati
real_path = "./results/ego_cut_kaggle/test_40/images/real_A"
fake_path = "./results/ego_cut_kaggle/test_40/images/fake_B"

# Trova immagini
images = glob.glob(os.path.join(real_path, "*.png")) + glob.glob(os.path.join(real_path, "*.jpg"))

if len(images) > 0:
    samples = random.sample(images, min(3, len(images)))
    
    plt.figure(figsize=(12, 8))
    for i, img_path in enumerate(samples):
        filename = os.path.basename(img_path)
        
        # Carica Reale e Sintetico
        img_real = Image.open(os.path.join(real_path, filename))
        img_fake = Image.open(os.path.join(fake_path, filename))
        
        # Mostra
        plt.subplot(3, 2, 2*i + 1)
        plt.imshow(img_real)
        plt.title("Input Reale")
        plt.axis('off')
        
        plt.subplot(3, 2, 2*i + 2)
        plt.imshow(img_fake)
        plt.title("Output Sintetico (CUT)")
        plt.axis('off')
    plt.show()
else:
    print("Nessuna immagine trovata nei risultati.")

In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/cut

OUTPUT_FOLDER = "./results/ego_cut_kaggle/test_40/images/fake_B"
ZIP_NAME = "dataset_transformed_fake_B.zip"

print(f"Comprimo la cartella {OUTPUT_FOLDER}...")
!zip -r {ZIP_NAME} {OUTPUT_FOLDER} -q

print(f"Zip pronto.")